# **Day_03**

## *Part 1/2*

### Instructions

--- Day 3: Crossed Wires ---

The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135

What is the Manhattan distance from the central port to the closest intersection?


### Imports

In [2]:
import pathlib

### Inputs

In [18]:
# Define file path for inputs
path = pathlib.Path("inputs/day_03.txt")

# Read and convert to list
diagram_raw = path.read_text().split("\n")

# Split out the wires to individual lists
wire_1 = diagram_raw[0].split(',')
wire_2 = diagram_raw[1].split(',')

### Solution

In [21]:
# Keep track of where on map we are
# Keep track of where on map we have been

# Perform for each wire

# Look at common 'where we have beens'

# Find closest common 'where we have beens'

In [39]:
# Set position to center
current_position = [0,0]

# Iterate through each wire segment
for ix,segment in enumerate(wire_1):
    print('='*20,f"Index {ix}",f"Segment {segment}",'='*20)
    
    direction = segment[0]
    distance = segment[1:]
    
    print(f"Dir:{direction} Dist:{distance}")
    
    # If up
    if direction == 'U':
        current_position[0] += int(distance)
    
    # If down
    elif direction == 'D':
        current_position[0] -= int(distance)
        
      # If down
    elif direction == 'L':
        current_position[1] -= int(distance)
        
    # If down
    elif direction == 'R':
        current_position[1] += int(distance)
        
    print(f"Current position is now {current_position}")

==================== Index 0 Segment R995 ====================
Dir:R Dist:995
Current position is now [0, 995]
==================== Index 1 Segment D882 ====================
Dir:D Dist:882
Current position is now [-882, 995]
==================== Index 2 Segment R144 ====================
Dir:R Dist:144
Current position is now [-882, 1139]
==================== Index 3 Segment U180 ====================
Dir:U Dist:180
Current position is now [-702, 1139]
==================== Index 4 Segment L638 ====================
Dir:L Dist:638
Current position is now [-702, 501]
==================== Index 5 Segment U282 ====================
Dir:U Dist:282
Current position is now [-420, 501]
==================== Index 6 Segment L907 ====================
Dir:L Dist:907
Current position is now [-420, -406]
==================== Index 7 Segment D326 ====================
Dir:D Dist:326
Current position is now [-746, -406]
==================== Index 8 Segment R731 ====================
Dir:R Dist:731
Current p

## *Part 2/2*

### Instructions

### Solution